In [19]:
import utils
import numpy as np

In [20]:
def region_growing(im: np.ndarray, seed_points: list, T: int) -> np.ndarray:
    """
        A region growing algorithm that segments an image into 1 or 0 (True or False).
        Finds candidate pixels with a Moore-neighborhood (8-connectedness).
        Uses pixel intensity thresholding with the threshold T as the homogeneity criteria.
        The function takes in a grayscale image and outputs a boolean image

        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
            seed_points: list of list containing seed points (row, col). Ex:
                [[row1, col1], [row2, col2], ...]
            T: integer value defining the threshold to used for the homogeneity criteria.
        return:
            (np.ndarray) of shape (H, W). dtype=bool
    """
    segmented = np.zeros_like(im).astype(bool)
    im = im.astype(float)
    for row, col in seed_points:
        segmented[row, col] = True

    def H(Ri, seed): #Homogeneity criteria helper function
        return abs(Ri - seed) < T

    def get_moore_neighborhood(im, row, col): #The Moore neighborhood helper function
        height, width = im.shape
        neighborhood = np.full(shape=(3,3), fill_value=(0,0,0), dtype=object) #Tuple containing value of image and i j indices

        pos_i = 0 #Counting where to place neighbor in neighborhood
        for i in range(row - 1, row + 2):
            pos_j = 0 #Counting where to place neighbor in neighborhood
            for j in range(col - 1, col + 2):
                if 0 <= i < height and 0 <= j < width: #Only include if within image
                    neighborhood[pos_i, pos_j] = (im[i, j], i, j) #Add value and position of neighbor
                pos_j += 1
            pos_i += 1

        return neighborhood

    #Looking at all the seed points
    for seed_point in seed_points:
        candidates = [seed_point]

        #Looping while candidates are present
        while len(candidates) > 0:
            #Get neighbors of candidate
            candidate = candidates.pop()
            neighbors = get_moore_neighborhood(im, candidate[0], candidate[1])

            #Loop through 3x3 neighbors
            for i in range(3):
                for j in range(3):
                    #Need to check if a valid neighbor first
                    if isinstance(neighbors[i,j], tuple):
                        #If passing homogeneity criteria and not already visited, then ...
                        if H(neighbors[i,j][0], im[seed_point[0],seed_point[1]]) and not segmented[neighbors[i,j][1],neighbors[i,j][2]]:
                            #... update segmented image and add more candidates to be checked
                            segmented[neighbors[i,j][1],neighbors[i,j][2]] = True
                            candidates.append([neighbors[i,j][1],neighbors[i,j][2]])


    return segmented
    ### END YOUR CODE HERE ###

In [21]:
if __name__ == "__main__":
    # DO NOT CHANGE
    im = utils.read_image("defective-weld.png")

    seed_points = [ # (row, column)
        [254, 138], # Seed point 1
        [253, 296], # Seed point 2
        [233, 436], # Seed point 3
        [232, 417], # Seed point 4
    ]
    intensity_threshold = 50
    segmented_image = region_growing(im, seed_points, intensity_threshold)

    assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
        im.shape, segmented_image.shape)
    assert segmented_image.dtype == bool, "Expected thresholded image dtype to be bool. Was: {}".format(
            segmented_image.dtype)

    segmented_image = utils.to_uint8(segmented_image)
    utils.save_im("defective-weld-segmented.png", segmented_image)

Reading image: images\defective-weld.png
Saving image to: image_processed\defective-weld-segmented.png
